In [1]:
import sys
import geopandas
from pathlib import Path
import os
import folium
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import aves

/tmp/ipykernel_4875/3769790492.py:2: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas


In [2]:
# se ajusta la memoria máxima a utilizar
# (por defecto usa el 80% de la memoria disponible)
sys.argv.append(["--max-memory", "2G"])

In [3]:
combined_route = "../web-scrapper/portalinmobiliario.com/data/gran_santiago_total/gransantiago_prices.csv" # reemplazar por el archivo definitivo
combined_df = pd.read_csv(combined_route)
combined_df.head()

,direccion,precio,gastos_comunes,caracteristicas,num_caracteristicas,fecha_descarga,fecha_publicacion,titulo,url,latitud,longitud,ciudad,precio_range
0,"Avenida Pedro De Valdivia 3300 - 3600, Ñuñoa, ...",690000.0,Gastos comunes aproximados $ 120.000,NaN,0,2023-06-13,Publicado hace 32 días,¡ Semi Amoblado ! Departamento 2d Y 2d (m) En ...,https://www.portalinmobiliario.com/MLC-1383721...,-33.455824,-70.605074,Santiago,"(500000.0, 1000000.0]"
1,"Santa Victoria 9300 - 9600, La Florida, Chile,...",650000.0,Gastos comunes aproximados $ 100.000,66 m² totales / 3 dormitorios / 2 baños,3,2023-06-13,Publicado hace 27 días,Departamento 3d Y 2b En Santa Victoria.,https://www.portalinmobiliario.com/MLC-1676897...,-33.537729,-70.554861,Santiago,"(500000.0, 1000000.0]"
2,"Avenida Irarrázaval 4800 - 5100, Ñuñoa, Chile,...",650000.0,Gastos comunes aproximados $ 130.000,60 m² totales / 2 dormitorios / 2 baños,3,2023-06-13,Publicado hace 2 meses,Departamento 2d Y 2b En Irarrazaval. Metro Vil...,https://www.portalinmobiliario.com/MLC-1430206...,-33.453552,-70.571432,Santiago,"(500000.0, 1000000.0]"
3,"Avenida Manquehue Norte 900 - 1200, Las Condes...",733000.0,Gastos comunes aproximados $ 150.000,45.91 m² totales / 1 dormitorio / 1 baño,3,2023-06-13,Publicado hace 11 meses,"Departamento Nuevo 1d1b, Insuperable Ubicación",https://www.portalinmobiliario.com/MLC-1359792...,-33.399006,-70.571399,Santiago,"(500000.0, 1000000.0]"
4,"Avenida Manquehue Norte 900 - 1200, Las Condes...",810000.0,Gastos comunes aproximados $ 125.000,48.09 m² totales / 1 dormitorio / 1 baño,3,2023-06-13,Publicado hace 1 año,"Departamento Nuevo 1d1b, Insuperable Ubicación",https://www.portalinmobiliario.com/MLC-1385350...,-33.399006,-70.571399,Santiago,"(500000.0, 1000000.0]"


In [6]:
# vamos a pasar los precios a int
#combined_df["precio"] = combined_df["precio"].replace(combined_df["precio"].values, list(map(replace_dot, combined_df["precio"].values)))

# y agregamos la columna id que será necesaria más tarde
combined_df["id"] = combined_df.index
combined_df['num_caracteristicas'] = pd.to_numeric(combined_df['num_caracteristicas'])
combined_df = combined_df[combined_df['num_caracteristicas'] != 0]
# Check if 'metros_cuadrados' column contains 'm² totales'
condition = combined_df['caracteristicas'].str.contains('m² totales')

# Filter 'df' based on the condition
combined_df = combined_df[condition]

combined_df['metros_cuadrados'] = combined_df['caracteristicas'].str.split(' m² totales').str[0]

# Convert 'metros_cuadrados' to numeric, setting invalid values to NaN
combined_df['metros_cuadrados'] = pd.to_numeric(combined_df['metros_cuadrados'], errors='coerce')

# Drop rows where 'metros_cuadrados' is NaN after conversion
combined_df.dropna(subset=['metros_cuadrados'], inplace=True)

combined_df.head()

,direccion,precio,gastos_comunes,caracteristicas,num_caracteristicas,fecha_descarga,fecha_publicacion,titulo,url,latitud,longitud,ciudad,precio_range,id,metros_cuadrados
1,"Santa Victoria 9300 - 9600, La Florida, Chile,...",650000.0,Gastos comunes aproximados $ 100.000,66 m² totales / 3 dormitorios / 2 baños,3,2023-06-13,Publicado hace 27 días,Departamento 3d Y 2b En Santa Victoria.,https://www.portalinmobiliario.com/MLC-1676897...,-33.537729,-70.554861,Santiago,"(500000.0, 1000000.0]",1,66.00
2,"Avenida Irarrázaval 4800 - 5100, Ñuñoa, Chile,...",650000.0,Gastos comunes aproximados $ 130.000,60 m² totales / 2 dormitorios / 2 baños,3,2023-06-13,Publicado hace 2 meses,Departamento 2d Y 2b En Irarrazaval. Metro Vil...,https://www.portalinmobiliario.com/MLC-1430206...,-33.453552,-70.571432,Santiago,"(500000.0, 1000000.0]",2,60.00
3,"Avenida Manquehue Norte 900 - 1200, Las Condes...",733000.0,Gastos comunes aproximados $ 150.000,45.91 m² totales / 1 dormitorio / 1 baño,3,2023-06-13,Publicado hace 11 meses,"Departamento Nuevo 1d1b, Insuperable Ubicación",https://www.portalinmobiliario.com/MLC-1359792...,-33.399006,-70.571399,Santiago,"(500000.0, 1000000.0]",3,45.91
4,"Avenida Manquehue Norte 900 - 1200, Las Condes...",810000.0,Gastos comunes aproximados $ 125.000,48.09 m² totales / 1 dormitorio / 1 baño,3,2023-06-13,Publicado hace 1 año,"Departamento Nuevo 1d1b, Insuperable Ubicación",https://www.portalinmobiliario.com/MLC-1385350...,-33.399006,-70.571399,Santiago,"(500000.0, 1000000.0]",4,48.09
5,"Avenida Manquehue Norte 900 - 1200, Las Condes...",835000.0,Gastos comunes aproximados $ 125.000,52.62 m² totales / 1 dormitorio / 1 baño,3,2023-06-13,Publicado hace 11 meses,"Departamento Nuevo 1d1b, Insuperable Ubicación",https://www.portalinmobiliario.com/MLC-1385225...,-33.399006,-70.571399,Santiago,"(500000.0, 1000000.0]",5,52.62


In [7]:
combined_df = combined_df[(combined_df.latitud <= -33.3) & (combined_df.latitud > -33.6) & (combined_df.longitud <= -70.4) & (combined_df.longitud > -70.8)]

In [8]:
from aves.features.geo import to_point_geodataframe

In [9]:
mycrs='epsg:32719'

In [ ]:
deptos = to_point_geodataframe(combined_df, 'longitud', 'latitud', crs=mycrs)
deptos.plot()

In [ ]:
precio_min = 300000
precio_max = 500000
min_metros_cuadrados = 45
max_metros_cuadrados = 70
origins = deptos[(deptos.precio >= precio_min) & (deptos.precio <= precio_max) & (deptos.metros_cuadrados >= min_metros_cuadrados) & (deptos.metros_cuadrados <= max_metros_cuadrados)]
origins = origins[["direccion", "latitud", "longitud", "id", "geometry"]]
origins

In [ ]:
# se hace un mapa para pasarlo como parámetro en la siguente celda
# esto para tener un punto específico de Santiago como centro y un nivel de zoom personalizados
mapa_stgo = folium.Map(location=(-33.45, -70.65), zoom_start=16)
mapa_stgo

In [ ]:


# Create a Folium map object
m1 = folium.Map(location=(-33.45, -70.65), zoom_start=16)  # Stgo coordinates



# Add markers to the map
for idx, row in deptos.iterrows():
    folium.CircleMarker(location=[row['latitud'], row['longitud']],
                        radius=5,
                        color='blue',
                        fill=True,
                        fill_color='blue').add_to(m1)

# Display the map
m1

In [ ]:
from shapely.geometry import Point, box
import geopandas as gpd

def create_grid(min_lat, max_lat, min_lon, max_lon, cell_size_lat, cell_size_lon):
    """Create a grid of polygons."""
    grid = []
    x_coords = [min_lon + (x * cell_size_lon) for x in range(int((max_lon - min_lon) / cell_size_lon) + 1)]
    y_coords = [min_lat + (y * cell_size_lat) for y in range(int((max_lat - min_lat) / cell_size_lat) + 1)]

    for x in range(len(x_coords) - 1):
        for y in range(len(y_coords) - 1):
            grid.append(box(x_coords[x], y_coords[y], x_coords[x+1], y_coords[y+1]))
    return grid

In [ ]:
# Define city boundaries (for Santiago de Chile)
min_lat = -33.6
max_lat = -33.3
min_lon = -70.8
max_lon = -70.4

# Grid resolution
cell_size_lat = 0.01
cell_size_lon = 0.01

grid = create_grid(min_lat, max_lat, min_lon, max_lon, cell_size_lat, cell_size_lon)
grid_gdf = gpd.GeoDataFrame({'geometry': grid})

# Create a spatial index
sindex = grid_gdf.sindex

# Example point to classify
point = Point(-70.65, -33.45)
# point = destino_2.iloc[0]["geometry"]

centroids = []
for i in origins.index:
    point = origins.loc[i]["geometry"]


    # Use spatial index to find the potential grid cell matches
    possible_matches_index = list(sindex.intersection(point.bounds))
    possible_matches = grid_gdf.iloc[possible_matches_index]
    #possible_matches = possible_matches.reset_index(drop=True)

    # Now refine possible matches to get the exact cell
    precise_match = possible_matches[possible_matches.intersects(point)]

    # Calcular centroides
    centroid = precise_match.centroid.iloc[0]

    centroids.append(centroid)

centroids_gdf = gpd.GeoDataFrame({'geometry': centroids})
centroids_gdf = centroids_gdf.drop_duplicates()
centroids_gdf = centroids_gdf.reset_index(drop=True)
centroids_gdf["id"] = centroids_gdf.index
centroids_gdf

In [ ]:
latitude = -33.467990
longitude = -70.593740
direccion_destino_2 = "Avenida Capitán Ignacio Carrera Pinto 21–57, Ñuñoa, Chile"
destino_df_2 = pd.DataFrame(
    {"direccion": direccion_destino_2,
     "latitud": latitude,
     "longitud": longitude},
     index=[14996]
)
destino_df_2["id"] = destino_df_2.index
destino_2 = to_point_geodataframe(destino_df_2, "longitud", "latitud", crs=my_crs)
destino_2

In [ ]:
# Create a Folium map object
m3= folium.Map(location=(-33.45, -70.65), zoom_start=13)  # Stgo coordinates
# Add a single point to the map
folium.CircleMarker(location=[float(destino_2["latitud"].iloc[0]), float(destino_2["longitud"].iloc[0])],
                    radius=5,
                    color='blue',
                    fill=True,
                    fill_color='blue').add_to(m3)
m3

In [21]:
from r5py import TransportNetwork

In [22]:
transport_network = TransportNetwork(
    "clipped-scl-osm.pbf", ["santiago_dptm_gtfs.zip"]
)

Running in non-interactive shell, SIGINT handler is replaced by shell
Signal Handlers:
SIGSEGV: [libjvm.so+0xbe53b0], sa_mask[0]=11111111011111111101111111111110, sa_flags=SA_RESTART|SA_SIGINFO
SIGBUS: [libjvm.so+0xbe53b0], sa_mask[0]=11111111011111111101111111111110, sa_flags=SA_RESTART|SA_SIGINFO
SIGFPE: [libjvm.so+0xbe53b0], sa_mask[0]=11111111011111111101111111111110, sa_flags=SA_RESTART|SA_SIGINFO
SIGPIPE: [libjvm.so+0xbe53b0], sa_mask[0]=11111111011111111101111111111110, sa_flags=SA_RESTART|SA_SIGINFO
SIGXFSZ: [libjvm.so+0xbe53b0], sa_mask[0]=11111111011111111101111111111110, sa_flags=SA_RESTART|SA_SIGINFO
SIGILL: [libjvm.so+0xbe53b0], sa_mask[0]=11111111011111111101111111111110, sa_flags=SA_RESTART|SA_SIGINFO
SIGUSR2: [libjvm.so+0xbe5250], sa_mask[0]=00000000000000000000000000000000, sa_flags=SA_RESTART|SA_SIGINFO
SIGHUP: [libjvm.so+0xbe5900], sa_mask[0]=11111111011111111101111111111110, sa_flags=SA_RESTART|SA_SIGINFO
SIGINT: [python+0xd0eeb], sa_mask[0]=000000000000000000000000

In [23]:
from r5py import TravelTimeMatrixComputer, TransitMode, LegMode

import datetime

In [24]:
travel_time_matrix_computer = TravelTimeMatrixComputer(
    transport_network,
    origins=origins,
    destinations=destino_2,
    departure=datetime.datetime(2023, 7, 3, 7, 0), # esta hora también la debería ingresar el usuario
    transport_modes=[TransitMode.TRANSIT, LegMode.WALK],
    breakdown=True
)

travel_time_matrix_detailed = travel_time_matrix_computer.compute_travel_times()
travel_time_matrix_detailed

/home/mprado/miniconda3/envs/aves/lib/python3.9/site-packages/r5py/r5/regional_task.py:394: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  self._regional_task.fromLat = origin.y
/home/mprado/miniconda3/envs/aves/lib/python3.9/site-packages/r5py/r5/regional_task.py:395: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  self._regional_task.fromLon = origin.x


,from_id,to_id,travel_time,routes,board_stops,alight_stops,ride_times,access_time,egress_time,transfer_time,wait_times,total_time,n_iterations
0,80,14544,48,"[517, L1, L2R, 506e]","[PC701, 85, 44, PA450]","[PC275, 44, 70, PA452]","[9.7, 7.2, 2.6, 2.6]",2.9,4.6,3.5,"[1.9, 2.0, 2.6, 1.5]",41.0,1


In [ ]:



# Assuming there's only one point in the GeoDataFrame for origin and destination
origin_point = origin.geometry.iloc[0]
destination_point = destino.geometry.iloc[0]

# Create a folium map centered on the origin point
m= folium.Map(location=[origin_point.y, origin_point.x], zoom_start=10)

# Draw a marker for the origin point
folium.Marker(
    location=[origin_point.y, origin_point.x],
    popup='Origin',
    icon=folium.Icon(color='blue')
).add_to(m)

# Draw a marker for the destination point
folium.Marker(
    location=[destination_point.y, destination_point.x],
    popup='Destination',
    icon=folium.Icon(color='red')
).add_to(m)

# Display the map
m

In [25]:
precio_min = 300000
precio_max = 500000
min_metros_cuadrados = 45
max_metros_cuadrados = 70
origins = deptos[(deptos.precio >= precio_min) & (deptos.precio <= precio_max) & (deptos.metros_cuadrados >= min_metros_cuadrados) & (deptos.metros_cuadrados <= max_metros_cuadrados)]
origins = origins[["direccion", "latitud", "longitud", "id", "geometry"]]
origins = origins.iloc[:5] # mostramos los primeros 100 resultados
origins

,direccion,latitud,longitud,id,geometry
1061,"Avenida María Rozas Velásquez 1 - 300, Santiag...",-33.456992,-70.705503,1061,POINT (-70.706 -33.457)
1062,"Rivera 1200 - 1500, Independencia, Chile, Hosp...",-33.425994,-70.657585,1062,POINT (-70.658 -33.426)
1070,"Av. Padre Alberto Hurtado 59, San Alberto Hurt...",-33.483102,-70.686329,1070,POINT (-70.686 -33.483)
1082,"Radal 66, Metro Ecuador, Estación Central, RM ...",-33.453893,-70.701903,1082,POINT (-70.702 -33.454)
1105,"Gaspar De Orense 897, Blanqueado, Quinta Norma...",-33.442136,-70.700048,1105,POINT (-70.700 -33.442)


In [27]:
m4 = folium.Map(location=(-33.45, -70.65), zoom_start=16)  # Stgo coordinates

# Add markers to the map
for idx, row in origins.iterrows():
    folium.CircleMarker(location=[row['latitud'], row['longitud']],
                        radius=5,
                        color='blue',
                        fill=True,
                        fill_color='blue').add_to(m4)

# Display the map
m4

In [ ]:
travel_time_matrix_computer = TravelTimeMatrixComputer(
    transport_network,
    origins=origins,
    destinations=destino,
    departure=datetime.datetime(2023, 7, 3, 7, 0), # esta hora también la debería ingresar el usuario
    transport_modes=[TransitMode.TRANSIT, LegMode.WALK],
    breakdown=True
)

travel_time_matrix_detailed = travel_time_matrix_computer.compute_travel_times()
travel_time_matrix_detailed

In [25]:

travel_time_matrix_computer_alldeptos = TravelTimeMatrixComputer(
    transport_network,
    origins=deptos,
    destinations=destino,
    departure=datetime.datetime(2023, 7, 3, 7, 0), # esta hora también la debería ingresar el usuario
    transport_modes=[TransitMode.TRANSIT, LegMode.WALK],
    breakdown=True
)

travel_time_matrix_detailed_alldeptos = travel_time_matrix_computer_alldeptos.compute_travel_times()
travel_time_matrix_detailed_alldeptos



/home/ignacio/miniconda3/envs/aves/lib/python3.9/site-packages/r5py/r5/regional_task.py:394: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  self._regional_task.fromLat = origin.y
/home/ignacio/miniconda3/envs/aves/lib/python3.9/site-packages/r5py/r5/regional_task.py:395: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  self._regional_task.fromLon = origin.x
/home/ignacio/miniconda3/envs/aves/lib/python3.9/site-packages/r5py/r5/regional_task.py:394: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  self._regional_task.fromLat = origin.y
/home/ignacio/miniconda3/envs/aves/lib/python3.9/site-packages/r5py/r5/regional_task.py:395: FutureWarning: Calling float on a single element Series is deprecated and wi

In [ ]:
print(type(travel_time_matrix_detailed_alldeptos))